**Question 3 -**<br>
Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%<br>
**Note -**
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively

**Ans**

In [1]:
# PyTorch Implementation

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

In [5]:
# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

100%|██████████| 9912422/9912422 [00:00<00:00, 86978657.37it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 39253303.25it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25218666.32it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7332767.04it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# Define the CNN model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = nn.functional.relu(self.conv1(x))
        out = nn.functional.max_pool2d(out, 2)
        out = nn.functional.relu(self.conv2(out))
        out = nn.functional.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = nn.functional.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [7]:
# Instantiate the model
model = PureCNN().to(device)

In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}")

Epoch [1/10], Step [100/469], Loss: 0.2155
Epoch [1/10], Step [200/469], Loss: 0.1197
Epoch [1/10], Step [300/469], Loss: 0.0905
Epoch [1/10], Step [400/469], Loss: 0.1175
Epoch [2/10], Step [100/469], Loss: 0.1786
Epoch [2/10], Step [200/469], Loss: 0.0389
Epoch [2/10], Step [300/469], Loss: 0.0321
Epoch [2/10], Step [400/469], Loss: 0.0313
Epoch [3/10], Step [100/469], Loss: 0.0296
Epoch [3/10], Step [200/469], Loss: 0.0137
Epoch [3/10], Step [300/469], Loss: 0.0244
Epoch [3/10], Step [400/469], Loss: 0.0505
Epoch [4/10], Step [100/469], Loss: 0.0817
Epoch [4/10], Step [200/469], Loss: 0.0303
Epoch [4/10], Step [300/469], Loss: 0.0204
Epoch [4/10], Step [400/469], Loss: 0.0549
Epoch [5/10], Step [100/469], Loss: 0.0164
Epoch [5/10], Step [200/469], Loss: 0.0224
Epoch [5/10], Step [300/469], Loss: 0.0254
Epoch [5/10], Step [400/469], Loss: 0.0492
Epoch [6/10], Step [100/469], Loss: 0.0282
Epoch [6/10], Step [200/469], Loss: 0.0369
Epoch [6/10], Step [300/469], Loss: 0.0135
Epoch [6/10

In [10]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.96%
